# Stock Prediction with SVM 

### Read company financials from sqlite database into Pandas Dataframe

In [1]:
# Import Python Pandas, Numpy and SQL toolkit
import pandas as pd
import sqlalchemy
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
import numpy as np

In [2]:
# Create engine using the `stock_data.sqlite` database file
engine = create_engine("sqlite:///stock_data.sqlite")

In [3]:
#Read all the data from the cmpny_financials table onto a data frame. Check size of the dataframe with shape()
sql = "select * from cmpny_financials"
company_data = pd.read_sql(sql, engine)
company_data.shape

(18732, 44)

In [4]:
company_data.head()

,id_cmpny_financials,id_cmpny,ticker,datekey,de,pe1,ps1,pb,netmargin,marketcap,...,payoutratio,tangibles,tbvps,workingcapital,price,sp_price,price_change,sp_price_change,diff,status
0,1,1.0,MMM,2008-05-02,1.153,15.048,2.221,4.449,0.153,5.532178e+10,...,0.357,2.121700e+10,30.031,4.625000e+09,78.55,1413.900024,0.000000,0.000000,0.000000,underperform
1,2,1.0,MMM,2008-08-01,1.183,13.261,1.925,3.912,0.140,4.903418e+10,...,0.370,2.059000e+10,29.326,3.552000e+09,70.15,1260.310059,-10.693826,-10.862859,0.169033,outperform
2,3,1.0,MMM,2008-10-31,1.263,11.952,1.722,3.652,0.151,4.455701e+10,...,0.350,2.077300e+10,29.868,3.780000e+09,64.30,968.750000,-18.141311,-31.483840,13.342529,outperform
3,4,1.0,MMM,2009-02-17,1.586,9.580,1.312,3.330,0.097,3.289960e+10,...,0.641,1.839600e+10,26.549,3.759000e+09,47.42,789.169983,-39.630808,-44.184881,4.554072,outperform
4,5,1.0,MMM,2009-05-01,1.458,13.429,1.680,4.128,0.102,4.019094e+10,...,0.680,1.740300e+10,25.094,3.919000e+09,57.88,877.520020,-26.314449,-37.936204,11.621755,outperform


In [5]:
#Delete index columns and columns with no data
del company_data["id_cmpny_financials"]
del company_data["id_cmpny"]

In [6]:
company_data = company_data[:5000]

In [7]:
#Fill empty entries & NaN to 0
company_data = company_data.fillna(0)

In [8]:
company_data.head()

,ticker,datekey,de,pe1,ps1,pb,netmargin,marketcap,ev,evebitda,...,payoutratio,tangibles,tbvps,workingcapital,price,sp_price,price_change,sp_price_change,diff,status
0,MMM,2008-05-02,1.153,15.048,2.221,4.449,0.153,5.532178e+10,5.877678e+10,8.701,...,0.357,2.121700e+10,30.031,4.625000e+09,78.55,1413.900024,0.000000,0.000000,0.000000,underperform
1,MMM,2008-08-01,1.183,13.261,1.925,3.912,0.140,4.903418e+10,5.349518e+10,7.867,...,0.370,2.059000e+10,29.326,3.552000e+09,70.15,1260.310059,-10.693826,-10.862859,0.169033,outperform
2,MMM,2008-10-31,1.263,11.952,1.722,3.652,0.151,4.455701e+10,4.935301e+10,7.146,...,0.350,2.077300e+10,29.868,3.780000e+09,64.30,968.750000,-18.141311,-31.483840,13.342529,outperform
3,MMM,2009-02-17,1.586,9.580,1.312,3.330,0.097,3.289960e+10,3.776860e+10,5.887,...,0.641,1.839600e+10,26.549,3.759000e+09,47.42,789.169983,-39.630808,-44.184881,4.554072,outperform
4,MMM,2009-05-01,1.458,13.429,1.680,4.128,0.102,4.019094e+10,4.459294e+10,7.812,...,0.680,1.740300e+10,25.094,3.919000e+09,57.88,877.520020,-26.314449,-37.936204,11.621755,outperform


### Use sklearn SVM classifier to predict stock performance based on company financials

In [9]:
# import sklearn dependencies
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm, preprocessing

In [10]:
#set status column as target
target = (company_data["status"]
            .replace("underperform", 0)
            .replace("outperform", 1))
target_names = ["underperform", "outperform"]

In [11]:
#Remove columns that are not company financials. This will be the features on which the model will be trained
data = (company_data.drop("price", axis=1)
            .drop("sp_price", axis=1)
            .drop("price_change", axis=1)
            .drop("sp_price_change", axis=1)
            .drop("diff", axis=1)
            .drop("status", axis=1)
            .drop("ticker", axis=1)
            .drop("datekey", axis=1))
feature_names = data.columns
data.head()

,de,pe1,ps1,pb,netmargin,marketcap,ev,evebitda,revenueusd,gp,...,ncfdebt,ncfcommon,ncfdiv,ncfx,ncf,sps,payoutratio,tangibles,tbvps,workingcapital
0,1.153,15.048,2.221,4.449,0.153,5.532178e+10,5.877678e+10,8.701,6.463000e+09,3.127000e+09,...,1.122000e+09,-431000000.0,-353000000.0,-17000000.0,8.310000e+08,35.369,0.357,2.121700e+10,30.031,4.625000e+09
1,1.183,13.261,1.925,3.912,0.140,4.903418e+10,5.349518e+10,7.867,6.739000e+09,3.229000e+09,...,-8.090000e+08,-434000000.0,-351000000.0,12000000.0,-1.180000e+09,36.441,0.370,2.059000e+10,29.326,3.552000e+09
2,1.263,11.952,1.722,3.652,0.151,4.455701e+10,4.935301e+10,7.146,6.558000e+09,3.126000e+09,...,1.181000e+09,-475000000.0,-348000000.0,-147000000.0,6.930000e+08,37.334,0.350,2.077300e+10,29.868,3.780000e+09
3,1.586,9.580,1.312,3.330,0.097,3.289960e+10,3.776860e+10,5.887,5.509000e+09,2.408000e+09,...,-4.570000e+08,-2000000.0,-346000000.0,-263000000.0,-3.910000e+08,36.140,0.641,1.839600e+10,26.549,3.759000e+09
4,1.458,13.429,1.680,4.128,0.102,4.019094e+10,4.459294e+10,7.812,5.089000e+09,2.317000e+09,...,-5.980000e+08,34000000.0,-354000000.0,13000000.0,-2.170000e+08,34.456,0.680,1.740300e+10,25.094,3.919000e+09


In [12]:
# There are some columns with value in billions. This has to be scaled inorder for the model to complete training
# Ignore the warning.
data = preprocessing.scale(data)

/Users/Bopanna/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [13]:
# Randomly split the data into training & testing data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [14]:
from sklearn.ensemble import RandomForestClassifier
# Create a random forest classifier
rf = RandomForestClassifier(n_estimators=200)

In [15]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
param_grid = { 
    'n_estimators': [200, 500, 700, 1000],
    'max_features': ['auto', 'sqrt', 'log2']
}

CV_rfc = GridSearchCV(estimator=rf, param_grid=param_grid, cv= 5)

In [16]:
CV_rfc.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [200, 500, 700, 1000], 'max_features': ['auto', 'sqrt', 'log2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [18]:
print(CV_rfc.best_params_)

{'max_features': 'sqrt', 'n_estimators': 700}


In [ ]:
# Model Accuracy
print('Test Acc: %.3f' % rf.score(X_train, y_train))

In [ ]:
# Model Accuracy
print('Test Acc: %.3f' % rf.score(X_test, y_test))

In [ ]:
# Random Forests in sklearn will automatically calculate feature importance
importances = rf.feature_importances_
importances

In [ ]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)